# Data Collection and Cleaning

In [1]:
import numpy as np
import pandas as pd
import duckdb
import matplotlib.pyplot as plt
import seaborn as sns

### Read Data from CSV Files

In [2]:
movies_data = pd.read_csv("data/movies_metadata.csv")
ratings_data = pd.read_csv("data/ratings_small.csv")

/var/folders/rj/8b1y14lj1rb1yqx9tb_cdshh0000gn/T/ipykernel_79788/2427516042.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_data = pd.read_csv("data/movies_metadata.csv")


In [3]:
print(movies_data.shape)
movies_data.head()

(45466, 24)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
print(ratings_data.shape)
ratings_data.head()

(100004, 4)


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


### Clean Data

In [5]:
# Filter to select only released movies
movies_data = movies_data[movies_data['status'] == 'Released']

In [6]:
# Drop irrelevant columns
movies_data.drop(columns=['adult', 'homepage', 'imdb_id', 'status', 'tagline',
                          'overview', 'poster_path', 'video', 'spoken_languages',
                          'production_companies', 'production_countries'], inplace=True)

In [7]:
# Convert release dates to datetime object
print(movies_data['release_date'].dtype)
movies_data['release_date'] = pd.to_datetime(movies_data['release_date'],
                                             format='mixed', errors='coerce')
print(movies_data['release_date'].dtype)

object
datetime64[ns]


In [8]:
# Convert budget from object to float
print(movies_data['budget'].dtype)
movies_data['budget'] = pd.to_numeric(movies_data['budget'], errors='coerce')
print(movies_data['budget'].dtype)

# Convert popularity from object to float
print(movies_data['popularity'].dtype)
movies_data['popularity'] = pd.to_numeric(movies_data['popularity'], errors='coerce')
print(movies_data['popularity'].dtype)

object
int64
object
float64


In [9]:
# Identify columns with missing data
print(movies_data.columns[movies_data.isnull().any()])

Index(['belongs_to_collection', 'original_language', 'release_date',
       'runtime'],
      dtype='object')


In [10]:
# Create dummy variable for whether or not the movie is part of a series
movies_data['belongs_to_collection'] = movies_data['belongs_to_collection'].notna()
movies_data['belongs_to_collection'] = movies_data['belongs_to_collection'].astype(int)
print(movies_data['belongs_to_collection'].head())

0    1
1    0
2    1
3    0
4    1
Name: belongs_to_collection, dtype: int64


In [11]:
# Drop rows with missing data
movies_data.dropna(inplace=True)
print(movies_data.shape)

(44686, 13)


In [12]:
# Filter out negative and 0 values for revenue and budget, as these may be incorrect data
movies_data = movies_data[(movies_data['revenue'] > 0) & (movies_data['budget'] > 0)]

In [13]:
# Create dummy variable for whether or not the original language is English
movies_data['original_language'] = movies_data['original_language'] == 'en'
movies_data['original_language'] = movies_data['original_language'].astype(int)
movies_data = movies_data.rename(columns={'original_language': 'original_english'})
print(movies_data['original_english'].head())

0    1
1    1
3    1
5    1
8    1
Name: original_english, dtype: int64


In [14]:
# Create dummy variable for whether or not the movie was released in the holiday season

# Extract month from release_date variable
movies_data['month'] = movies_data['release_date'].dt.month
movies_data['month'] = movies_data['month'].astype(int)
print(movies_data['month'].head())

# Holiday months are December-February and June-August, the rest are non-holiday
movies_data['holiday'] = (
    ((movies_data['month'] >= 6) & (movies_data['month'] <= 8)) | 
    ((movies_data['month'] == 12) | (movies_data['month'] <= 2)))
movies_data['holiday'] = movies_data['holiday'].astype(int)
print(movies_data['holiday'].head())

0    10
1    12
3    12
5    12
8    12
Name: month, dtype: int64
0    0
1    1
3    1
5    1
8    1
Name: holiday, dtype: int64


In [15]:
# Find average of ratings for each movie
ratings_data = duckdb.sql("SELECT movieId, AVG(rating) AS rating_average, \
COUNT(rating) AS rating_count FROM ratings_data GROUP BY movieId").df()
ratings_data.head()

,movieId,rating_average,rating_count
0,31,3.178571,42
1,1263,3.864583,48
2,2455,3.393617,47
3,52,3.637255,51
4,110,3.945175,228


### Create Joined Dataset

In [16]:
# Merge the data frames
data = duckdb.sql("SELECT * FROM movies_data INNER JOIN ratings_data \
ON movies_data.id = ratings_data.movieId").df()
print(data.shape)
data.drop(columns=['movieId'], inplace=True)
data.head()

(1030, 18)


,belongs_to_collection,budget,genres,id,original_english,original_title,popularity,release_date,revenue,runtime,title,vote_average,vote_count,month,holiday,rating_average,rating_count
0,0,40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",508,1,Love Actually,10.586815,2003-09-07,244931766.0,135.0,Love Actually,7.0,1917.0,9,0,3.668605,86
1,0,42000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",509,1,Notting Hill,9.874043,1999-05-13,363889678.0,124.0,Notting Hill,7.0,1309.0,5,0,3.750000,78
2,1,65000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",1271,1,300,18.108408,2006-12-09,422610419.0,117.0,300,7.0,5092.0,12,1,3.700000,40
3,0,24000000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",1089,1,Point Break,11.453093,1991-07-12,83531958.0,120.0,Point Break,6.9,861.0,7,1,4.162879,132
4,1,30000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",2770,1,American Pie 2,10.251030,2001-08-10,287553595.0,108.0,American Pie 2,6.0,1375.0,8,1,3.068627,51


In [17]:
data.to_csv('data/data.csv', index=False)